# Multi-Agent Systems: Calling Agents As Tools

Welcome to this multi-agent lab! Here we'll explore the supervisor pattern, where one agent coordinates the work of other specialized agents.

## Why Use Multiple Agents?
Specialized agents perform better at specific tasks than a single agent trying to do everything. The supervisor agent delegates work to these specialists and manages the overall process.

## What We'll Build
In module 2, we used LangGraph so it seems fitting to pick a different framework (PydanticAI) to see how other frameworks work. We'll use PydanticAI to create a multi-agent system that can answer questions and debug an open search cluster. 

We'll create 3 agents:
* A supervisor agent that manages all the other agents
* A Researcher agent that has access to the internet and can research topics
* A Writer agent that can take the information retrieved from the internet and create a report

At the end of this module we should be on our way to creating a "Deep Research" Agentic system.

# Create our Researcher Agent
Let's create a researcher agent that can create a research plan and get information from the internet to create a summary of the research. First lets double check that the instructions in 0_setup.ipynb were set up correctly and import our Search key. 

In [1]:
import os 
from dotenv import load_dotenv
# Check if you created the .env file before running this notebook.
print('Does the .env file exist?', os.path.exists('.env'))
# from dotenv import load_dotenv
load_dotenv('.env')

Does the .env file exist? True


True

<div style="background-color: #fffbe6; border-left: 4px solid #faad14; padding: 15px; margin-bottom: 20px;">
<strong>⚠️ Warning</strong><br>
If the command above doesn't say .env exists is true then you need to complete the 0_setup.ipynb notebook first to grab your tavily API key.
</div>

In [2]:
# We need to use nest_asyncio which patches the asyncio to allow nested event loops which PydanticAI runs on.
import nest_asyncio
nest_asyncio.apply()

# Build our Search Agent
In this section we'll build an agent that can search the web and get results back (and annotations) to answer some tough questions. 

## A note on search
We'll build the tool from scratch to demonstrate it's use but in practice there's a number of ways to DRY (Don't repeat yourself). Pydantic ships with a tavily tool you can just import. Tavily also has an MCP server available that you can use as well. The only reason you would write this yourself is if the default tool or MCP server doesn't output the results in the way you want. 

In [3]:
from pydantic import BaseModel
from typing import Dict
from agentic_platform.core.models.memory_models import ToolResult
from agentic_platform.core.models.prompt_models import BasePrompt
from tavily import TavilyClient
#  We should start aliasing the agent class to avoid conflict with Agent objects from other frameworks. 
from pydantic_ai import Agent as PyAIAgent

# Now lets create our research tools
class WebSearch(BaseModel):
    query: str

def search_web(query: WebSearch) -> ToolResult:
    '''Search the web to get back a list of results and content.'''
    client: TavilyClient = TavilyClient(os.getenv("TAVILY_KEY"))
    response: Dict[str, any] = client.search(query=query.query)

    return ToolResult(
        content=[
            {'type': 'json', 'content': response['results']}
        ],
        isError=False
    )




In [4]:
# Create prompt using our BasePrompt class. 
SYSTEM_PROMPT = """
You are a specialized Research Agent with web search capabilities. Your role is to:

1. Analyze user queries and construct a question to query the internet with. 
2. Organize findings into comprehensive, well-sourced research briefs
3. Return the research brief in a well structured format that a writer can use to write a report.
4. Make sure to cite your sources with links in markdown format at the bottom of the research brief.

Provide only the research based of your web search results in a format that a writer can use to write a report.
Make sure to cite your sources with links in markdown format at the bottom of the research brief.
"""

USER_PROMPT = """
Using the users research question as context, do all the research needed to answer the question.
Remember, you are not writing the report. You are only providing the research needed to write the report.
"""

class ResearchPlanPrompt(BasePrompt):
    system_prompt: str = SYSTEM_PROMPT
    user_prompt: str = USER_PROMPT

research_prompt: BasePrompt = ResearchPlanPrompt()

research_agent: PyAIAgent = PyAIAgent(
    'bedrock:us.anthropic.claude-3-sonnet-20240229-v1:0',
    system_prompt=SYSTEM_PROMPT,
)

# Add our search tool to the agent.
research_agent.tool_plain(search_web)

<function __main__.search_web(query: __main__.WebSearch) -> agentic_platform.core.models.memory_models.ToolResult>

In [ ]:
# Test the researcher agent individually
research_question: str = "What's different between how the US and the EU handle pasturization of milk?"

result = research_agent.run_sync(research_question)

# Store this here to test our writer agent.
research_brief = result.output

print(result.output)
print(result.usage())


# Create our Writer Agent
This agent will take in the research and be responsible for drafting the results of the response. It doesn't need any tools, but we do want it to write quickly so we'll use a smaller model to take the research and formulate a response.

In [6]:
# Create prompt using our BasePrompt class. 
WRITER_SYSTEM_PROMPT = """
You are a specialized Writer Agent responsible for crafting polished, cohesive reports from research provided by the Research Agent. Your role is to:

1. Transform the short research brief into a comprehensive report.
2. Organize information logically with appropriate sections and flow
3. Maintain a professional, authoritative tone appropriate for the subject matter
4. Ensure clarity, conciseness, and readability for the target audience

You will be provided with comprehensive research materials that include facts, figures, and sourced information. Your job is to synthesize this information without altering facts or adding unsupported claims.

Please use complete sentenences and paragraphs. No bullet points. Break up the report into section with headers with the following format:
Title:
[Title of the report]

Section 1:
[Section 1 of the report]

Conclusion:
[Conclusion of the report]
"""

WRITER_USER_PROMPT = """
Using the research and users question provided as context, please write a comprehensive report addressing the following query.
"""

class WriterPrompt(BasePrompt):
    system_prompt: str = WRITER_SYSTEM_PROMPT
    user_prompt: str = WRITER_USER_PROMPT

writer_prompt: BasePrompt = WriterPrompt()

# Lets use a really fast model for the writer agent.
writer_agent: PyAIAgent = PyAIAgent(
    'bedrock:us.anthropic.claude-3-5-haiku-20241022-v1:0',
    system_prompt=writer_prompt.system_prompt
)

In [ ]:
# Test the writer agent individually
user_query: str = """What's different between how the US and the EU handle pasturization of milk?

Research Brief:
{research_brief}
"""

user_query = user_query.format(research_brief=research_brief)
results = writer_agent.run_sync(user_query)

print(results.output)
print(results.usage())

# Create Supervisor Agent
Lastly, we need to create the supervisor agent. This agent is responsible for delegating work to it's workers. In Pydantic there's a couple ways to do this. For this experiment we will delegate using tools. The supervisor agent will have two tools that invoke the writer agent and researcher agent. 

In [8]:
SUPERVISOR_SYSTEM_PROMPT = """
You are a Supervisor Agent responsible for coordinating a multi-agent research system. Your role is to:

1. Analyze user queries and coordinate the research and writing process
2. Delegate specific tasks to the Research Agent and Writer Agent
3. Ensure the final report fully addresses the user's query
4. Return the Writer Agent's final report directly to the user without modification in <writer_output> tags.

You must NOT analyze, summarize, or modify the Writer Agent's output. Your role is purely to coordinate the process and deliver the complete, unaltered report from the Writer Agent to the user.

If the Writer Agent requests additional information, you should coordinate with the Research Agent to provide it, then return to the Writer Agent to complete the report.
When the writer agent is done, copy the output and return it to the user.

If the research isn't giving you exactly what you need, that's okay. Please don't call it multiple times.
Remember, take the writers output and return it to the user in <writer_output> tags.
"""

SUPERVISOR_USER_PROMPT = """
Please help with the following query:
{user_query}

Your task is to:
1. Coordinate with the Research Agent to gather necessary information
2. Pass that research to the Writer Agent to create a final report
3. Return the Writer Agent's complete report directly to the user without any additional commentary

Do not add your own analysis or summarize the report - simply return the Writer Agent's complete output.
"""

class SupervisorPrompt(BasePrompt):
    system_prompt: str = SUPERVISOR_SYSTEM_PROMPT
    user_prompt: str = SUPERVISOR_USER_PROMPT


# supervisor_prompt: BasePrompt = SupervisorPrompt()

supervisor_agent: PyAIAgent = PyAIAgent(
    'bedrock:us.anthropic.claude-3-sonnet-20240229-v1:0',
    system_prompt=SUPERVISOR_SYSTEM_PROMPT,
)

# Bring everything together
In Pydantic, you can pass the run context to tools. For our web search tool, it doesn't need the run context so we added the tool as tool_plain. For agents as tools, it needs the context so we'll set it up as a tool that takes in the run context.

In [9]:
from pydantic_ai import RunContext

@supervisor_agent.tool
async def research_agent_tool(ctx: RunContext[None]) -> str:
    '''Useful for researching a topic using the internet'''

    # Useful to see what's going on. Please don't use print statements in production :) 
    print('entering research agent tool')
    results = await research_agent.run(  
        research_prompt.user_prompt,
        usage=ctx.usage,  
    )
    return results.output 

@supervisor_agent.tool
async def writer_agent_tool(ctx: RunContext[None], research_brief: str) -> str:
    '''Useful for writing a report on a research topic provided the research is done.'''
    
    # Useful to see what's going on. Please don't use print statements in production :) 
    print('Entering the writer agent tool')

    prompt = f'write a report on the following research brief: {research_brief}'
    results = await writer_agent.run(
        prompt,
        usage=ctx.usage,  
    )
    return results.output 


In [ ]:

research_question: str = "What's different between how the US and the EU handle pasturization of milk?"

result = supervisor_agent.run_sync(research_question)
print(result.output)
print(result.usage())

# Analysis
The multi-agent system is coordinating with a researcher and a writer to generate the final report for the user. However it's not the most efficient way of doing things. For one, the supervisor is verbatim copying the output of the writer agent verbatim. Secondly, there's no guardrails around how much the supervisor calls the researcher aside from asking it to not call it multiple times. 

This is a pretty naive implementation of this. In a production system, you'd probably want to either return directly from the writer agent or perform some more complex agent<>agent communication with things like reference passing. 

# Conclusion
In this lab we coordinated 3 agents together to answer questions and write reports using a SERPs API. In the next lab we'll coordinate agents using a Graph Structure. Graph orchestration is a much more practical and flexible approach to multi agent collaboration